In [2]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Fungsi untuk load data
@st.cache_data
def load_data():
    try:
        # Gunakan relative path atau pastikan file ada di folder yang benar
        day_df = pd.read_csv(r'D:\DBS\submission\data\day.csv')
        hour_df = pd.read_csv(r'D:\DBS\submission\data\hour.csv')
        return day_df, hour_df
    except FileNotFoundError:
        st.error("File dataset tidak ditemukan. Pastikan file 'day.csv' dan 'hour.csv' ada di direktori yang sama.")
        return None, None

# Fungsi untuk visualisasi tren peminjaman berdasarkan hari
def plot_daily_trend(day_df):
    fig, ax = plt.subplots(figsize=(10, 6))
    day_weekday_agg = day_df.groupby('weekday')['cnt'].mean().reset_index()
    sns.barplot(x='weekday', y='cnt', data=day_weekday_agg, ax=ax)
    ax.set_title('Rata-rata Peminjaman Sepeda Berdasarkan Hari')
    ax.set_xlabel('Hari')
    ax.set_ylabel('Rata-rata Jumlah Peminjaman')
    return fig

# Fungsi untuk visualisasi distribusi peminjaman berdasarkan jam
def plot_hourly_distribution(hour_df):
    fig, ax = plt.subplots(figsize=(10, 6))
    hour_agg = hour_df.groupby('hr')['cnt'].mean().reset_index()
    sns.barplot(x='hr', y='cnt', data=hour_agg, ax=ax)
    ax.set_title('Rata-rata Peminjaman Sepeda Berdasarkan Jam')
    ax.set_xlabel('Jam')
    ax.set_ylabel('Rata-rata Jumlah Peminjaman')
    return fig

# Fungsi untuk visualisasi korelasi suhu dan peminjaman
def plot_temperature_correlation(hour_df):
    fig, ax = plt.subplots(figsize=(10, 6))
    # Normalisasi suhu dan jumlah peminjaman untuk memudahkan interpretasi
    hour_df['temp_normalized'] = (hour_df['temp'] - hour_df['temp'].min()) / (hour_df['temp'].max() - hour_df['temp'].min())
    hour_df['cnt_normalized'] = (hour_df['cnt'] - hour_df['cnt'].min()) / (hour_df['cnt'].max() - hour_df['cnt'].min())
    
    # Scatter plot dengan warna berdasarkan kepadatan
    scatter = ax.scatter(hour_df['temp_normalized'], hour_df['cnt_normalized'], 
                         c=hour_df['cnt'], cmap='viridis', alpha=0.6)
    ax.set_title('Korelasi Suhu dengan Jumlah Peminjaman Sepeda')
    ax.set_xlabel('Suhu (Dinormalisasi)')
    ax.set_ylabel('Jumlah Peminjaman (Dinormalisasi)')
    plt.colorbar(scatter, ax=ax, label='Jumlah Peminjaman')
    return fig

# Halaman Utama Streamlit
def main():
    st.title('Dashboard Analisis Data Bike Sharing')
    
    # Load data
    day_df, hour_df = load_data()
    
    if day_df is not None and hour_df is not None:
        # Sidebar untuk navigasi
        page = st.sidebar.selectbox('Pilih Halaman', 
                                    ['Tren Peminjaman', 
                                     'Distribusi Jam Peminjaman', 
                                     'Korelasi Suhu', 
                                     'Kesimpulan'])
        
        # Tampilan berdasarkan halaman yang dipilih
        if page == 'Tren Peminjaman':
            st.header('Tren Peminjaman Sepeda Berdasarkan Hari')
            fig = plot_daily_trend(day_df)
            st.pyplot(fig)
            
            # Tambahkan fitur interaktif untuk memilih hari
            selected_day = st.selectbox('Pilih Hari:', day_df['weekday'].unique())
            day_detail = day_df[day_df['weekday'] == selected_day]
            st.write(f"Detail Peminjaman untuk Hari {selected_day}:")
            st.dataframe(day_detail[['cnt', 'casual', 'registered', 'temp', 'atemp']])
        
        elif page == 'Distribusi Jam Peminjaman':
            st.header('Distribusi Peminjaman Sepeda Berdasarkan Jam')
            fig = plot_hourly_distribution(hour_df)
            st.pyplot(fig)
            
            # Tambahkan slider untuk filter jam
            selected_hour = st.slider('Pilih Rentang Jam:', 
                                      min_value=0, 
                                      max_value=23, 
                                      value=(0, 23))
            hour_filtered = hour_df[(hour_df['hr'] >= selected_hour[0]) & 
                                    (hour_df['hr'] <= selected_hour[1])]
            st.write(f"Total Peminjaman pada Jam {selected_hour[0]} - {selected_hour[1]}:")
            st.write(hour_filtered['cnt'].sum())
        
        elif page == 'Korelasi Suhu':
            st.header('Korelasi Suhu dengan Jumlah Peminjaman')
            fig = plot_temperature_correlation(hour_df)
            st.pyplot(fig)
            
            # Hitung korelasi
            correlation = hour_df['temp'].corr(hour_df['cnt'])
            st.write(f"Korelasi antara Suhu dan Jumlah Peminjaman: {correlation:.2f}")
        
        elif page == 'Kesimpulan':
            st.header('Kesimpulan Analisis Bike Sharing')
            st.markdown("""
            ### Temuan Utama Analisis Data Bike Sharing

            #### 1. Tren Peminjaman Berdasarkan Hari
            - Terdapat variasi signifikan dalam jumlah peminjaman sepeda antar hari
            - Hari kerja cenderung memiliki peminjaman lebih tinggi dibandingkan akhir pekan

            #### 2. Distribusi Peminjaman Berdasarkan Jam
            - Dua puncak peminjaman utama:
              * Pagi hari (sekitar pukul 7-9)
              * Sore hari (sekitar pukul 17-19)
            - Pola ini mencerminkan kebiasaan komuter

            #### 3. Pengaruh Suhu terhadap Peminjaman
            - Korelasi positif antara suhu dan jumlah peminjaman
            - Cuaca yang lebih hangat mendorong peningkatan penggunaan sepeda

            ### Implikasi Praktis
            1. Strategi penyebaran sepeda perlu mempertimbangkan pola harian dan jam sibuk
            2. Pertimbangkan musim dan suhu dalam manajemen armada sepeda
            3. Fokus pada jam-jam sibuk untuk optimasi layanan
            """)

# Jalankan aplikasi
if __name__ == '__main__':
    main()

ModuleNotFoundError: No module named 'streamlit'